In [16]:
import requests
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000
import numba
import numpy
def patch_asscalar(a):

    return a.item()

setattr(numpy, "asscalar", patch_asscalar)

In [15]:
!pip install line_profiler numba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 5.9 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 4.8 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [17]:
sampleFeatureSet = requests.get("http://dsapostgisapi:82/getTileFeatures?imageId=6452a8cb239ec54642356ce8&ftxtract_id=54")

referenceColor = [242,243,243]
distanceThresh = 5
featureSet= sampleFeatureSet.json()

In [3]:
%load_ext line_profiler

In [21]:
%prun  computeColorSimilarityForFeatureSet(featureSet,referenceColor,distanceThresh)

/tmp/ipykernel_65/3367826529.py:2: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "computeColorSimilarityForFeatureSet" failed type inference due to: Untyped global name 'sRGBColor': Cannot determine Numba type of <class 'type'>

File "../../tmp/ipykernel_65/3367826529.py", line 9:
<source missing, REPL/exec in use?>

  @jit
/tmp/ipykernel_65/3367826529.py:2: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "computeColorSimilarityForFeatureSet" failed type inference due to: Untyped global name 'sRGBColor': Cannot determine Numba type of <class 'type'>

File "../../tmp/ipykernel_65/3367826529.py", line 9:
<source missing, REPL/exec in use?>

  @jit
/usr/local/lib/python3.9/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "computeColorSimilarityForFeatureSet" was compiled in object mode without forceobj=True, but has lifted loops.

File "../../tmp/ipykernel

In [ ]:
# sampleTileTwo = sampleFeatureSet.json()[55]
# c2_avg = sampleTileTwo['average']

# color2_rgb = sRGBColor(c2_avg[0],c2_avg[1],c2_avg[2],is_upscaled=True)
# color2_lab = convert_color(color2_rgb, LabColor)


In [9]:
# delta_e = delta_e_cie2000(color1_lab, color2_lab);

# print("The difference between the 2 color = ", delta_e)

In [29]:
from numba import jit, njit

def computeColorSimilarityForFeatureSet( featureSet, referenceColor, distanceThreshold):
    ### This expects a featureset, in this case it will simply be the average color for each tile
    ## I also am passing the referenceColor (i.e. feature) separately... this should then go through each individual
    ## feature and convert it to LAB space and use the colormath library to compute the delta_e score
    ## In the futurue I may want to look into cython or HIT
    """ referenceColor: [Array]"""
    refColor_rgb = sRGBColor(referenceColor[0], referenceColor[1],referenceColor[2], is_upscaled=True)
    refColor_lab = convert_color(refColor_rgb, LabColor)
    
    tilesInRange = [] ### This will store the tiles that are within the specified unit distance and also their ID
    for i in featureSet:
        tileColor_rgb = sRGBColor(i['average'][0],i['average'][1],i['average'][2],is_upscaled=True)
        tileColor_lab = convert_color(tileColor_rgb, LabColor)
        delta_e = delta_e_cie2000(refColor_lab, tileColor_lab)
        tilesInRange.append((i['localTileId'],delta_e))
        
    return tilesInRange
    

In [85]:
%%time

def recomputeColorSimilarity(featureSet, referenceColorRGB, distanceThreshold):
    refColor_lab = rgb2lab(referenceColorRGB)
    print(refColor_lab)
    tilesInRange = []
    for i in featureSet:
        tileColor_lab = rgb2lab(i['average'])
        delta_e = deltaE(refColor_lab,tileColor_lab)
        if delta_e < distanceThresh:
            tilesInRange.append((i['localTileId'],delta_e))
    return tilesInRange
rs = recomputeColorSimilarity(featureSet,referenceColor,distanceThresh)


[95.53404442366264, -0.34514377171568356, -0.1332120757263633]
CPU times: user 77.3 ms, sys: 213 µs, total: 77.5 ms
Wall time: 74.9 ms


In [58]:
rs

[('0', 0.023601611666919986),
 ('1', 0.04663481275766924),
 ('2', 0.017908298142847178),
 ('3', 0.0),
 ('4', 0.0),
 ('5', 0.0),
 ('6', 0.0),
 ('7', 0.0),
 ('8', 0.0034007859878661477),
 ('9', 0.0),
 ('10', 0.0),
 ('11', 0.0),
 ('12', 0.317194241108564),
 ('13', 0.0),
 ('14', 0.0),
 ('15', 0.0),
 ('16', 0.0),
 ('17', 0.0),
 ('18', 0.0),
 ('19', 0.00271497408077633),
 ('20', 0.037130532538436574),
 ('21', 0.0),
 ('22', 0.011021930318367495),
 ('23', 0.00794221347799492),
 ('24', 0.06608642649459902),
 ('25', 0.11847672450575164),
 ('26', 0.3214540685358194),
 ('27', 0.4515041100265383),
 ('28', 0.01990922786743137),
 ('29', 0.010109578448923571),
 ('30', 0.0),
 ('31', 0.0),
 ('32', 0.0),
 ('33', 0.0),
 ('34', 0.019894100595180386),
 ('35', 0.011657627178652168),
 ('36', 0.02203695758698499),
 ('37', 0.12366163314963838),
 ('38', 0.23271725801365326),
 ('39', 0.09586317827439102),
 ('40', 0.2871446997819388),
 ('41', 0.0027551190533337284),
 ('42', 0.35478206005936425),
 ('43', 0.40347702

In [30]:
%%time
_ = computeColorSimilarityForFeatureSet(featureSet,referenceColor,distanceThresh)


CPU times: user 2.41 s, sys: 11.8 ms, total: 2.42 s
Wall time: 2.41 s


In [75]:
# @jit(nopython=True)
def lab2rgb(lab):
    y = (lab[0] + 16) / 116
    x = lab[1] / 500 + y
    z = y - lab[2] / 200
    r, g, b = 0, 0, 0

    x = 0.95047 * (x ** 3 if x ** 3 > 0.008856 else (x - 16/116) / 7.787)
    y = 1.00000 * (y ** 3 if y ** 3 > 0.008856 else (y - 16/116) / 7.787)
    z = 1.08883 * (z ** 3 if z ** 3 > 0.008856 else (z - 16/116) / 7.787)

    r = x *  3.2406 + y * -1.5372 + z * -0.4986
    g = x * -0.9689 + y *  1.8758 + z *  0.0415
    b = x *  0.0557 + y * -0.2040 + z *  1.0570

    r = (1.055 * r ** (1/2.4) - 0.055) if r > 0.0031308 else 12.92 * r
    g = (1.055 * g ** (1/2.4) - 0.055) if g > 0.0031308 else 12.92 * g
    b = (1.055 * b ** (1/2.4) - 0.055) if b > 0.0031308 else 12.92 * b

    return [
        max(0, min(1, r)) * 255,
        max(0, min(1, g)) * 255,
        max(0, min(1, b)) * 255
    ]


# # Red Color
# color1_rgb = sRGBColor(1.0, 0.0, 0.0);
# @jit(nopython=True)
def rgb2lab(rgb):
    r = rgb[0] / 255
    g = rgb[1] / 255
    b = rgb[2] / 255
    x, y, z = 0, 0, 0

    r = (r ** 2.4 if r > 0.04045 else r / 12.92)
    g = (g ** 2.4 if g > 0.04045 else g / 12.92)
    b = (b ** 2.4 if b > 0.04045 else b / 12.92)

    x = (r * 0.4124 + g * 0.3576 + b * 0.1805) / 0.95047
    y = (r * 0.2126 + g * 0.7152 + b * 0.0722) / 1.00000
    z = (r * 0.0193 + g * 0.1192 + b * 0.9505) / 1.08883

    x = x ** (1/3) if x > 0.008856 else (7.787 * x) + 16/116
    y = y ** (1/3) if y > 0.008856 else (7.787 * y) + 16/116
    z = z ** (1/3) if z > 0.008856 else (7.787 * z) + 16/116

    return [
        (116 * y) - 16,
        500 * (x - y),
        200 * (y - z)
    ]


import math
# @jit(nopython=True)
def deltaE(labA, labB):
    deltaL = labA[0] - labB[0]
    deltaA = labA[1] - labB[1]
    deltaB = labA[2] - labB[2]
    c1 = math.sqrt(labA[1] * labA[1] + labA[2] * labA[2])
    c2 = math.sqrt(labB[1] * labB[1] + labB[2] * labB[2])
    deltaC = c1 - c2
    deltaH = deltaA * deltaA + deltaB * deltaB - deltaC * deltaC
    deltaH = 0 if deltaH < 0 else math.sqrt(deltaH)
    sc = 1.0 + 0.045 * c1
    sh = 1.0 + 0.015 * c1
    deltaLKlsl = deltaL / 1.0
    deltaCkcsc = deltaC / sc
    deltaHkhsh = deltaH / sh
    i = deltaLKlsl * deltaLKlsl + deltaCkcsc * deltaCkcsc + deltaHkhsh * deltaHkhsh
    return 0 if i < 0 else math.sqrt(i)


In [ ]:


# # Blue Color
# color2_rgb = sRGBColor(0.0, 0.0, 1.0);

# # Convert from RGB to Lab Color Space
# color1_lab = convert_color(color1_rgb, LabColor);

# # Convert from RGB to Lab Color Space
# color2_lab = convert_color(color2_rgb, LabColor);

# # Find the color difference
# delta_e = delta_e_cie2000(color1_lab, color2_lab);

# print("The difference between the 2 color = ", delta_e)